# Spatially varying parameters - Mark

## Objectives

The main objective of this tutorial is to demonstrate how spatially varying parameters can be used to model and simulate different geometries and/or materials using JOOMMF. I have modified this notebook in order to simulate a structure similar to the core-shell particle from Anderson and Camley paper.

## Problem specification

The geometry is a **sphere** with

- unit cell length $\Delta=0.228 \,\text{nm}$
- core radius $r_c = 21\cdot\Delta= 4.8\,\text{nm}$ 
- radius $r = 31\cdot\Delta= 6.5\,\text{nm}$ 
- domain length $l = 100 \,\text{nm}$.

The material parameters for the core are (Gd):

- exchange energy constant $A = 1.3 \times 10^{-11} \,\text{J/m}$,
- magnetisation saturation $M_\text{s} = 8 \times 10^{5} \,\text{A/m}$.

The material parameters for the shell are (Fe):

- exchange energy constant $A = 1.3 \times 10^{-11} \,\text{J/m}$,
- magnetisation saturation $M_\text{s} = 8 \times 10^{5} \,\text{A/m}$.

Magnetisation dynamics are governed by the Landau-Lifshitz-Gilbert equation

$$\frac{d\mathbf{m}}{dt} = \underbrace{-\gamma_{0}(\mathbf{m} \times \mathbf{H}_\text{eff})}_\text{precession} + \underbrace{\alpha\left(\mathbf{m} \times \frac{d\mathbf{m}}{dt}\right)}_\text{damping}$$

where $\gamma_{0} = 2.211 \times 10^{5} \,\text{m}\,\text{A}^{-1}\,\text{s}^{-1}$ and Gilbert damping $\alpha=0.5$.

We are interested in computing the equlibrium magnetisation state starting from the uniform magnetisation in the $(1, 1, 1)$ direction.

## Simulation

In the first step, we import the required `discretisedfield` and `oommfc` modules.

In [ ]:
#import hublib.use
import numpy as np
#%use oommf-1.2b0

import oommfc as oc
import discretisedfield as df
mu0 = 1.25663706212e-6

We need to define the rectangular finite difference mesh that can contain the entire sphere.

In [ ]:
n_1 = 21 # number of cells for inner core
n_i = 2 # number of cells in interface
n_2 = 10 # number of cells in outer shell
d = 0.228e-9  # discretisation cell (m)
#L = 20e-9  # mesh edge length (m)
L = 3*(n_1 + n_i + n_2)*d
mesh = oc.Mesh(p1=(-L/2, -L/2, -L/2), p2=(L/2, L/2, L/2), cell=(d, d, d))

To illustrate the mesh and discretisation cell:

In [ ]:
%matplotlib inline
mesh

As usual, we create the system object and define its Hamiltonian and dynamics equation.

In [ ]:
import math
#System object
system = oc.System(name="sphere")
# Hamiltonian
#A = 1.3e-11  # exchange energy constant (J/m)
H = (0, 0, 0.2e-3/mu0)  # external magnetic field (A/m)
#
# Here we define the layered geometry
#
A_1 = 2.78e-11
A_i = 8.78e-11
A_2 = 5.11e-11
M_1 = 8e5
M_i = -4e5
M_2 = 5e5
def A_value(pos):
    x, y, z = pos
    r = math.sqrt(x**2 + y**2 + z**2)
    if r <= n_1 * d: # inner core
        return A_1
    elif n_1 * d < r and (n_1 + n_i) * d <= r:
        return A_i
    elif (n_1 + n_i) * d <= r and (n_1 + n_i + n_2) * d < r:
        return A_2    
    else:
        return 0
    
def Ms_function(pos):
    x, y, z = pos
    r = math.sqrt(x**2 + y**2 + z**2)
    if r <= n_1 * d: # inner core
        return M_1
    elif n_1 * d < r and (n_1 + n_i) * d <= r:
        return M_i
    elif (n_1 + n_i) * d <= r and (n_1 + n_i + n_2) * d < r:
        return M_2    
    else:
        return 0   
    
#ex = df.Field(mesh, dim=1, value=A_value)
#ex.Region


#system.m = df.Field(mesh, value=(1, 0, 0), norm=Ms_function)
#print(type(system.m))
#oc.Exchange(system.m)
#system.m = df.Field(mesh, value=(1, 0, 0), norm=Ms_function)
#type(ex)
#type(system.m)
A =  df.Field(mesh, dim=1, value=A_value)
    
system.hamiltonian = oc.Exchange(A=A) + oc.Demag() + oc.Zeeman(H)

## Dynamics
#gamma = 2.211e5
#alpha = 0.5
#system.dynamics = oc.Precession(gamma) + oc.Damping(alpha)

The Hamiltonian and dynamics equation are:

In [ ]:
system.hamiltonian

In [ ]:
system.dynamics

Having the function defining the sphere geometry, we can now create the initial magnetisation field. The direction of the inital field is determined by the value variable, here $(1, 0, 0)$, i.e., the inital magnetization points into the $x$-direction and the norm should be the saturation magnetisation function we just created.

In [ ]:
system.m = df.Field(mesh, value=(1, 0, 0), norm=Ms_function)

We can plot the initial magnetisation by slicing the cylinder geometry perpendicular to "z" and "x" axes.

In [ ]:
#system.m.norm.k3d_nonzero()
system.m.plane('x').k3d_vectors(head_size=10)
#system.m.plot_plane("z")
#system.m.plot_plane("x")

### Relaxing the magnetisation

After we defined all requited parameters of the system, we can relax the system using `MinDriver`.

In [ ]:
md = oc.MinDriver()
md.drive(system, overwrite=True)

And plot the magnetisation in the same slices as before.

In [ ]:
system.m.z.k3d_voxels(norm_field=system.m.norm)

<hr>
Cleaning up.  Temporary files were written.  You may want to remove them.

In [ ]:
!rm -rf cylinder